# Basic Pandas & NumPy code optimization
#### ~ Little hacks for major GAINZ ~

**SUMMARY**

**A) Go underground**
- Pandas DataFrame and Series are built on top of NumPy arrays
- Operating on the underlying NumPy array (accessible using `.values` and `.to_numpy()` methods), using NumPy methods (`np.sum(s)` instead of `s.sum()`), and choosing positional instead of boolean indexing (`np.where(condition)`), compounds to 4x, 10x, **even 100x faster** execution of certain operations.

**B) Categories as Categories**
- Storing your categorical data as strings in DataFrames is savagery. Use the `'categorical'` data type (`astype('category')`), available both in Pandas and NumPy, for major GAINZ in indexing/caounting/grouping/aggregating! Works even faster than with numerical encoding, proof below!

**C) Exceptions, as always**
- pandas still wins by a large margin when it comes to finding unique values of a Series (`pd.Series.unique()` and `pd.Series.value_counts()`), where it's 2 - 200x times faster than the numpy equivalent (`np.unique()`)

### Initialize test data

In [530]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis

In [531]:
nrows = 10**6

df = pd.DataFrame()
df['dec'] = np.random.normal(size=nrows)*100                  # random decimal numbers
df['int'] = (np.random.uniform(size=nrows)*100).astype('int') # random integers (int format)
df['int_float'] = df.int.astype('float')                      # random integers (float format)
df['cat_str'] = np.random.choice(['A', 'B', 'C'], size=nrows) # random categorical var (string format)
df['cat_cat'] = df.cat_str.astype('category')                 # random categorical var (category format)
df['cat_num'] = df.cat_cat.cat.codes                          # random categorical var (int format)

In [532]:
df.head()

,dec,int,int_float,cat_str,cat_cat,cat_num
0,0.790252,74,74.0,B,B,1
1,-206.060663,40,40.0,A,A,0
2,-120.401291,89,89.0,C,C,2
3,37.482814,83,83.0,A,A,0
4,-1.567344,48,48.0,A,A,0


In [502]:
df.dtypes

dec           float64
int             int64
int_float     float64
cat_str        object
cat_cat      category
cat_num          int8
dtype: object

# INDEXING: Boolean and positional

In [501]:
# Input: Random decimal numbers
%timeit df.dec[np.round(df.dec*0.01) == 1]
%timeit df.dec.values[np.round(df.dec*0.01) == 1]
%timeit df.dec.values[np.where(np.round(df.dec*0.01) == 1)]
%timeit df.dec.values[np.where(np.round(df.dec.values*0.01) == 1)]

14.5 ms ± 196 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
9.67 ms ± 116 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
9.17 ms ± 152 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.2 ms ± 142 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [395]:
# Input: Categorical values, data type: STRING
%timeit df.cat_str[df.cat_str == 'A']
%timeit df.cat_str.values[df.cat_str == 'A']
%timeit df.cat_str.values[np.where(df.cat_str == 'A')]
%timeit df.cat_str.values[np.where(df.cat_str.values == 'A')]

52.4 ms ± 4.74 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
45.7 ms ± 296 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
42 ms ± 95.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
17.1 ms ± 124 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [396]:
# Input: Categorical values, data type: CATEGORY
%timeit df.cat_cat[df.cat_cat == 'A']
%timeit df.cat_cat.values[df.cat_cat == 'A']
%timeit df.cat_cat.values[np.where(df.cat_cat == 'A')]
%timeit df.cat_cat.values[np.where(df.cat_cat.values == 'A')]

26.3 ms ± 219 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
4.96 ms ± 8.02 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.88 ms ± 9.24 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.61 ms ± 31 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [397]:
# Input: Categorical values, data type: INTEGER
%timeit df.cat_num[df.cat_num == 1]
%timeit df.cat_num.values[df.cat_num == 1]
%timeit df.cat_num.values[np.where(df.cat_num == 1)]
%timeit df.cat_num.values[np.where(df.cat_num.values == 1)]

9.52 ms ± 65.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.85 ms ± 18.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.74 ms ± 4.43 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.56 ms ± 6.88 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Unique values

In [503]:
# Input: Integers as INT
%timeit np.unique(df.int.values)
%timeit df.int.unique()

61.7 ms ± 7.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
3.71 ms ± 164 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [504]:
# Input: Integers as FLOAT (.0) ==> visibly slower
%timeit np.unique(df.int_float.values)
%timeit df.int_float.unique()

69 ms ± 4.28 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
12.4 ms ± 113 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [505]:
# Input: Categorical as STR
%timeit np.unique(df.cat_str.values)
%timeit df.cat_str.unique()

506 ms ± 31.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
24.1 ms ± 244 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [506]:
# Input: Categorical as CATEGORY ==> MUCH faster!
%timeit np.unique(df.cat_cat.values)
%timeit df.cat_cat.unique()

487 ms ± 10.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.15 ms ± 133 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [507]:
# Input: Categorical as INT ==> Only slightly faster than CATEGORY for PANDAS; visibly faster for Numpy
%timeit np.unique(df.cat_num.values)
%timeit df.cat_num.unique()

16 ms ± 94.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.65 ms ± 24.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Value counts

In [513]:
# Input: Integers as INT
%timeit np.unique(df.int.values, return_counts=True)
%timeit df.int.value_counts()

59.5 ms ± 4.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
9.04 ms ± 1.71 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [514]:
# Input: Integers as FLOAT (.0) ==> visibly slower
%timeit np.unique(df.int_float.values, return_counts=True)
%timeit df.int_float.value_counts()

75.1 ms ± 2.77 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
21.4 ms ± 4.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [515]:
# Input: Categorical as STR
%timeit np.unique(df.cat_str.values, return_counts=True)
%timeit df.cat_str.value_counts()

492 ms ± 21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
48.4 ms ± 186 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [516]:
# Input: Categorical as CATEGORY ==> MUCH faster (10x)
%timeit np.unique(df.cat_cat.values, return_counts=True)
%timeit df.cat_cat.value_counts()

455 ms ± 2.27 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.35 ms ± 25.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [517]:
# Input: Categorical as INT ==> STRANGELY, much slower (4x) than CATEGORY counting using the Pandas method.
%timeit np.unique(df.cat_num.values, return_counts=True)
%timeit df.cat_num.value_counts()

16.5 ms ± 463 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
9.01 ms ± 1.04 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


# The advantage of "category" type

In [518]:
%timeit df.cat_str.value_counts()
%timeit df.cat_num.value_counts()
%timeit df.cat_cat.value_counts()

47.3 ms ± 542 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
8.16 ms ± 648 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.43 ms ± 34.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Min / Max / Mean / Median / Std / Sum /...

In [519]:
%timeit min(df.dec.values)
%timeit np.min(df.dec)
%timeit df.dec.min()
%timeit np.min(df.dec.values) # Numpy 10x faster than Pandas!

89.6 ms ± 1.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
4.08 ms ± 395 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.38 ms ± 113 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
321 µs ± 2.12 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [520]:
%timeit max(df.dec.values)
%timeit np.max(df.dec)
%timeit df.dec.max()
%timeit np.max(df.dec.values)

86.3 ms ± 748 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
3.77 ms ± 265 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.49 ms ± 242 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
393 µs ± 5.59 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [416]:
%timeit df.num.mean()
%timeit np.mean(df.num)
%timeit np.mean(df.num.values)

1.16 ms ± 5.46 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.15 ms ± 7.85 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
318 µs ± 210 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [417]:
%timeit np.median(df.num)
%timeit np.median(df.num.values)
%timeit df.num.median()

12.2 ms ± 349 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
11.9 ms ± 12.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
9.32 ms ± 10.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [420]:
%timeit df.num.std()
%timeit np.std(df.num)
%timeit np.std(df.num.values)

2.23 ms ± 3.11 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.24 ms ± 3.78 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.59 ms ± 3.32 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [421]:
%timeit sum(df.num)
%timeit df.num.sum()
%timeit np.sum(df.num)
%timeit np.sum(df.num.values)

53.7 ms ± 353 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
3.3 ms ± 250 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.34 ms ± 150 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
263 µs ± 784 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [134]:
%timeit sub = kurtosis(X2)
%timeit sub = kurtosis(X2.values)
%timeit sub = X2.kurtosis()

print('-'*80)

%timeit sub = skew(X2)
%timeit sub = skew(X2.values)
%timeit sub = X2.skew()

9.17 ms ± 22.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
9.03 ms ± 12.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
15.8 ms ± 137 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
--------------------------------------------------------------------------------
10.4 ms ± 8.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
10.3 ms ± 24.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
16.3 ms ± 156 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [422]:
%timeit np.percentile(X2, [10, 25, 75, 90])
%timeit np.percentile(X2.values, [10, 25, 75, 90])

18.4 ms ± 334 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
19.1 ms ± 1.24 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Length

In [423]:
%timeit len(df.num)
%timeit df.num.shape[0]
%timeit df.num.values.shape[0]
%timeit len(df.num.values)

3.82 µs ± 19.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
3.65 µs ± 24.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
3.49 µs ± 11.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
3.47 µs ± 6.75 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [424]:
%timeit abs(df.num)
%timeit np.abs(df.num)
%timeit np.abs(df.num.values)

646 µs ± 53 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
627 µs ± 1.49 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
534 µs ± 833 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Group By

In [523]:
%timeit df.dec.groupby(df.cat_str).sum()
%timeit df.dec.groupby(df.cat_num).sum()
%timeit df.dec.groupby(df.cat_cat).sum()

32.2 ms ± 269 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
5.8 ms ± 67.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
11.1 ms ± 52 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Groupping/Splitting

In [533]:
%timeit [df_part for k, df_part in df.groupby('cat_str')]
%timeit [df_part for k, df_part in df.groupby('cat_cat')]

72.9 ms ± 5.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
51.7 ms ± 262 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
47.4 ms ± 3.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [535]:
%timeit [df[df.cat_str == lbl] for lbl in ['A', 'B', 'C']]
%timeit [df[df.cat_str.values == lbl] for lbl in ['A', 'B', 'C']]

%timeit [df[df.cat_cat == lbl] for lbl in ['A', 'B', 'C']]
%timeit [df[df.cat_cat.values == lbl] for lbl in ['A', 'B', 'C']]

160 ms ± 3.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
82.5 ms ± 397 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
48.4 ms ± 136 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
46.5 ms ± 106 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
47.3 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
46.5 ms ± 189 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# BONUS: Accelertion using Numba

**NOTE:** You cannot do this simply with any function: It needs to be written using "pure" NumPy and considering many restrictions that Numba imposes in order to "digest it" (numba doesn't play along well with strings, for example).

Still, for "purely numerical" functions, numba's "just-in-time" compiler can provide amazing boosts.

In [556]:
import numba

def normal_decimal_extractor(X, n_decimals):
    
    r = np.floor((X - np.floor(X))*(10**n_decimals))

    return r 
        
fast_decimal_extractor = numba.njit(normal_decimal_extractor)

In [561]:
%timeit normal_decimal_extractor(df.dec.values, 2)
%timeit fast_decimal_extractor(df.dec.values, 2) # runs faster after the 1st run.

3.01 ms ± 10.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
618 µs ± 3.61 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
